# Obrada slike


## Potrebne knjižnice

In [1]:
import javax.imageio.ImageIO
import java.io.File
import java.io._
import java.nio.file.{Files,Paths}

import org.opencv.core._
import org.opencv.highgui.Highgui;
import org.opencv.imgproc.Imgproc;

import com.sksamuel.scrimage._
import com.sksamuel.scrimage.nio._

import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.clustering.{KMeans, KMeansModel}



nu.pattern.OpenCV.loadShared()


Otvaranje slike sa scrimage i skaliranje na određenu dimenziju

In [2]:
val inputStream = new FileInputStream("0001.jpg")
val image = Image.fromStream(inputStream).scaleTo(300, 150)
image.output(new File("scaled.jpeg"))

scaled.jpeg

![title](0001.jpg)

![title](scaled.jpeg)

## Uklanjanje prozirnosti


In [3]:
val opaque = image.map((x,y,pixel) => new Pixel((pixel.argb & 0x00ffffff) + (0xff<<24)))
image.output(new File("opaque.jpeg"))

opaque.jpeg

## Jednostavna segmentacija slike

In [3]:
val inputStream = new FileInputStream("jump.jpg")
val image = Image.fromStream(inputStream)

val segmentation = image.map{ (x,y,pixel) => 
        
        if(pixel.blue > 65) new Pixel(0xff000000) else  new Pixel(0xffffffff)
    }
segmentation.output(new File("segmentation.jpeg"))

segmentation.jpeg

![title](jump.jpg)

![title](segmentation.jpeg)

## Slika u crno-bijelom formatu

In [5]:
val image = Highgui.imread("0001.jpg",  0);

Highgui.imwrite("blackAndWhite.jpeg", image);


true

![title](blackAndWhite.jpeg)

## Edge detection

In [6]:
val image =  Highgui.imread("street.jpeg",  Highgui.CV_LOAD_IMAGE_COLOR);
val dest = new Mat()

Imgproc.Canny(image, dest, 20, 100, 3, true);
Highgui.imwrite("edge.jpeg", dest);

true

![title](street.jpeg)

![title](edge.jpeg)


## Morfološka obrada slike

In [2]:
val image = Highgui.imread("edge.jpeg",  Highgui.CV_LOAD_IMAGE_COLOR);

val destination = new Mat()

val element = Imgproc.getStructuringElement(Imgproc.MORPH_RECT, new  Size(2, 2));
Imgproc.dilate(image, destination, element);
Highgui.imwrite("erode.jpeg", destination);

true

![title](erode.jpeg)


In [8]:
val hsvImage = new Mat()
Imgproc.cvtColor(image, hsvImage, Imgproc.COLOR_BGR2HSV);
println(hsvImage)

Mat [ 478*506*CV_8UC3, isCont=true, isSubmat=false, nativeObj=0x7f9a242d2f70, dataAddr=0x7f9a2451aa30 ]


## Učitavanje svih slika u byte array

In [9]:
val imageArray =  new java.io.File("/home/teo/dataset/").listFiles.
    map{ file => 
        
        try { 
            Files.readAllBytes(Paths.get(file.toString))
        }
        catch {
              case e : Exception => Array() : Array[Byte]
        }
    }

In [10]:
val in = new FileInputStream("/home/teo/dataset/0001.jpg")
val image = Image.fromStream(in).scaleTo(10, 5).pixels.foreach(println)


Pixel(-3949118)
Pixel(-6911085)
Pixel(-6778986)
Pixel(-6712679)
Pixel(-6580582)
Pixel(-6514787)
Pixel(-6580067)
Pixel(-6514017)
Pixel(-6316895)
Pixel(-3158319)
Pixel(-4343878)
Pixel(-9410195)
Pixel(-10658465)
Pixel(-7764854)
Pixel(-6975342)
Pixel(-6975341)
Pixel(-7172205)
Pixel(-7237484)
Pixel(-7303275)
Pixel(-3684663)
Pixel(-7106159)
Pixel(-11054766)
Pixel(-12566208)
Pixel(-13156542)
Pixel(-6581101)
Pixel(-5857379)
Pixel(-6580588)
Pixel(-6843243)
Pixel(-7960951)
Pixel(-4144958)
Pixel(-7435913)
Pixel(-12305375)
Pixel(-11056076)
Pixel(-11778771)
Pixel(-12172501)
Pixel(-12105932)
Pixel(-11382719)
Pixel(-10002606)
Pixel(-10462379)
Pixel(-5395543)
Pixel(-7436180)
Pixel(-12700926)
Pixel(-12174324)
Pixel(-11845358)
Pixel(-12173551)
Pixel(-12370417)
Pixel(-12173294)
Pixel(-11647461)
Pixel(-11450849)
Pixel(-5528431)


In [12]:
val imageRdd = sc.parallelize(imageArray)

val pixelsRdd = imageRdd.map{ x => 
    try{
            ImageReader.fromBytes(x).scaleTo(5,5).pixels.map(x => x.argb.toDouble)}
    catch{
            case e : Exception => Array() : Array[Double]
    }
}

In [14]:
println(pixelsRdd)

MapPartitionsRDD[1] at map at <console>:43


In [15]:
println("Prvi element")

val vectorsRdd = pixelsRdd.map(x => Vectors.dense(x))

println(vectorsRdd.first)

Prvi element
[-5265747.0,-6910315.0,-6580581.0,-6645860.0,-4606021.0,-6844012.0,-9343374.0,-6909548.0,-7369327.0,-5329487.0,-9079950.0,-1.2631228E7,-6712173.0,-6777965.0,-5855575.0,-9607344.0,-1.1713493E7,-1.2172241E7,-1.0988474E7,-7632765.0,-9805509.0,-1.230591E7,-1.2239088E7,-1.217355E7,-8226211.0]
